In [ ]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

In [ ]:
#loading data and preprocessing

# Define the transformation for preprocessing the data

# define transforms for images
transform = transforms.Compose(
              [transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the CIFAR-100 training and test datasets

cifar10_train = CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar10_test = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=32, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torchvision.models as models
# Load the EfficientNetB0 model
effnet = models.efficientnet_b0(pretrained=True)

# Remove the last layer (classifier)
effnet.classifier = torch.nn.Identity()

In [ ]:
# define gausian noise
class GaussianNoiseLayer(nn.Module):
    def __init__(self, std_dev):
        super(GaussianNoiseLayer, self).__init__()
        self.std_dev = std_dev
    def forward(self, x):
        noise = torch.randn_like(x) * self.std_dev
        return x + noise

# define communication model with std 0.1
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.gaussian_noise = GaussianNoiseLayer(std_dev=0.1)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )


    def forward(self, x):
        x = self.encoder(x)
        x = (8*8*8)*x/torch.norm(x)
        x = self.gaussian_noise(x)
        x = self.decoder(x)

        return x


In [ ]:
import os

# Get the current working directory
current_path = os.getcwd()

# Load the trained weights
path=current_path+'/comm_model_std01.pt'

comm_model1  = torch.load(path)

comm_model1.to(device)

Autoencoder(
  (gaussian_noise): GaussianNoiseLayer()
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(16, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): Tanh()
  )
)

In [ ]:
# Freeze the parameters of the communication model
for param in comm_model1.parameters():
    param.requires_grad = False

In [ ]:
class Cifar10ClassfierModel(nn.Module):
    def __init__(self, effnet, num_classes):
        super(Cifar10ClassfierModel, self).__init__()
        self.effnet = effnet
        self.relu = nn.ReLU()
        self.batch_norm = nn.BatchNorm2d(1280)
        self.dropout = nn.Dropout(p=0.5)
        self.dense1 = nn.Linear(1280, 640)
        self.dense2 = nn.Linear(640, 320)
        self.dense3 = nn.Linear(320, 160)
        self.dense4 = nn.Linear(160, 10)
        self.dense = nn.Linear(1280, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.effnet(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.dense4(x)
        # x = self.sigmoid(x)
        return x

In [ ]:
num_classes = 10  # Replace with the desired number of output classes
classifier = Cifar10ClassfierModel(effnet, num_classes)

# Set the model to evaluation mode
classifier.train()

# Print the modified model architecture
print(classifier)

classifier.to(device)

In [ ]:
# Define the optimizer and the initial learning rate
optimizer = optimizer = optim.Adam(classifier.parameters(), lr=0.0001)

# Define the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=70, gamma=0.1)

# Define the loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
# train classifier on noisy image passed through channel with gausian noise 0.1
num_epochs = 20
for epoch in range(num_epochs):
    # Adjust the learning rate
    classifier.train()
    scheduler.step()
    total = 0
    correct = 0
    # Train the model for one epoch
    for images, labels in train_loader:
        optimizer.zero_grad()
        labels = labels.to(device)
        images = images.to(device)
        n_images = comm_model1(images)
        n_images.to(device)
        outputs = classifier(n_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    accuracy = correct / total
    # Print the loss after each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()},Accuracy:{100 * accuracy:.2f}%")

    # Change learning rate after 25 epochs
    if epoch == 70:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.00001

print('Finished training')

# Evaluate the model on the test set
classifier.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        n_images = comm_model1(images)
        n_images.to(device)
        outputs = classifier(n_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Epoch [1/20], Loss: 1.7136214971542358,Accuracy:44.26%
Epoch [2/20], Loss: 0.8692020177841187,Accuracy:62.03%
Epoch [3/20], Loss: 1.3526527881622314,Accuracy:67.18%
Epoch [4/20], Loss: 1.1034802198410034,Accuracy:71.02%
Epoch [5/20], Loss: 0.9938765168190002,Accuracy:73.91%
Epoch [6/20], Loss: 0.6535024642944336,Accuracy:76.25%
Epoch [7/20], Loss: 0.7966820001602173,Accuracy:78.02%
Epoch [8/20], Loss: 0.9075096845626831,Accuracy:79.68%
Epoch [9/20], Loss: 0.6211148500442505,Accuracy:81.05%
Epoch [10/20], Loss: 0.43409982323646545,Accuracy:82.88%
Epoch [11/20], Loss: 0.5064211487770081,Accuracy:83.89%
Epoch [12/20], Loss: 0.5413274765014648,Accuracy:84.82%
Epoch [13/20], Loss: 0.3358219861984253,Accuracy:85.80%
Epoch [14/20], Loss: 1.7257121801376343,Accuracy:87.40%
Epoch [15/20], Loss: 0.6910967230796814,Accuracy:87.79%
Epoch [16/20], Loss: 0.29653626680374146,Accuracy:88.66%
Epoch [17/20], Loss: 0.9794277548789978,Accuracy:89.59%
Epoch [18/20], Loss: 0.21595746278762817,Accuracy:90.03

In [ ]:
# Evaluate the model on the test set based on image pased through channel with gausian noise 0.1
classifier.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        n_images = comm_model1(images)
        n_images.to(device)
        outputs = classifier(n_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Accuracy on the test set: 79.24 %


In [ ]:
# define communication model with std 0.6
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.gaussian_noise = GaussianNoiseLayer(std_dev=0.6)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )


    def forward(self, x):
        x = self.encoder(x)
        x = (8*8*8)*x/torch.norm(x)
        x = self.gaussian_noise(x)
        x = self.decoder(x)

        return x



In [ ]:
import os

# Get the current working directory
current_path = os.getcwd()

# Load the trained weights
path=current_path+'/comm_model_std06.pt'

comm_model6  = torch.load(path)

comm_model6.to(device)

Autoencoder(
  (gaussian_noise): GaussianNoiseLayer()
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(16, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): Tanh()
  )
)

In [ ]:
# Evaluate the model (trained with gausian noise with std 0.1) on the test set based on image pased through channel with gausian noise 0.6
classifier.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        n_images = comm_model6(images)
        n_images.to(device)
        outputs = classifier(n_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Accuracy on the test set: 68.37 %


In [ ]:
# define communication model with std 0.0
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.gaussian_noise = GaussianNoiseLayer(std_dev=0.0)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )


    def forward(self, x):
        x = self.encoder(x)
        x = (8*8*8)*x/torch.norm(x)
        x = self.gaussian_noise(x)
        x = self.decoder(x)

        return x



In [ ]:
import os

# Get the current working directory
current_path = os.getcwd()

# Load the trained weights
path=current_path+'/comm_model_std00.pt'

comm_model0  = torch.load(path)

comm_model0.to(device)

Autoencoder(
  (gaussian_noise): GaussianNoiseLayer()
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(16, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): Tanh()
  )
)

In [ ]:
# Evaluate the model (trained with gausian noise with std 0.1) on the test set based on image pased through channel with gausian noise 0.0
classifier.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        n_images = comm_model0(images)
        n_images.to(device)
        outputs = classifier(n_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Accuracy on the test set: 73.89 %


In [ ]:
classifier2 = Cifar10ClassfierModel(effnet, num_classes)

# Set the model to evaluation mode
classifier2.train()

# Print the modified model architecture
print(classifier2)

classifier2.to(device)

In [ ]:
# train classifier on noisy image passed through channel with gausian noise 0.6
num_epochs = 20
for epoch in range(num_epochs):
    # Adjust the learning rate
    classifier2.train()
    scheduler.step()
    total = 0
    correct = 0
    # Train the model for one epoch
    for images, labels in train_loader:
        optimizer.zero_grad()
        labels = labels.to(device)
        images = images.to(device)
        n_images = comm_model6(images)
        n_images.to(device)
        outputs = classifier2(n_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    accuracy = correct / total
    # Print the loss after each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()},Accuracy:{100 * accuracy:.2f}%")

    # Change learning rate after 25 epochs
    if epoch == 70:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.00001

print('Finished training')

# Evaluate the model on the test set
classifier2.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        n_images = comm_model6(images)
        n_images.to(device)
        outputs = classifier2(n_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Epoch [1/20], Loss: 2.137986660003662,Accuracy:21.02%
Epoch [2/20], Loss: 1.8213571310043335,Accuracy:44.76%
Epoch [3/20], Loss: 1.8431624174118042,Accuracy:56.95%
Epoch [4/20], Loss: 1.8054453134536743,Accuracy:63.40%
Epoch [5/20], Loss: 1.8738234043121338,Accuracy:68.01%
Epoch [6/20], Loss: 1.9457108974456787,Accuracy:69.92%
Epoch [7/20], Loss: 2.0031895637512207,Accuracy:71.74%
Epoch [8/20], Loss: 1.7076064348220825,Accuracy:73.32%
Epoch [9/20], Loss: 1.5045374631881714,Accuracy:74.49%
Epoch [10/20], Loss: 1.7739430665969849,Accuracy:75.47%
Epoch [11/20], Loss: 1.6848034858703613,Accuracy:76.00%
Epoch [12/20], Loss: 1.4785118103027344,Accuracy:77.15%
Epoch [13/20], Loss: 1.376766324043274,Accuracy:78.28%
Epoch [14/20], Loss: 1.3885983228683472,Accuracy:79.12%
Epoch [15/20], Loss: 1.448828935623169,Accuracy:80.05%
Epoch [16/20], Loss: 1.4075216054916382,Accuracy:81.15%
Epoch [17/20], Loss: 1.6227729320526123,Accuracy:81.49%
Epoch [18/20], Loss: 1.3016983270645142,Accuracy:82.21%
Epoc

In [ ]:
# Evaluate the model on the test set based on image pased through channel with gausian noise 0.6
classifier2.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        n_images = comm_model6(images)
        n_images.to(device)
        outputs = classifier2(n_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Accuracy on the test set: 75.93 %


In [ ]:
# Evaluate the model (trained with gausian noise with std 0.6) on the test set based on image pased through channel with gausian noise 0.1
classifier2.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        n_images = comm_model0(images)
        n_images.to(device)
        outputs = classifier2(n_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Accuracy on the test set: 77.06 %
